In [1]:
from osgeo import gdal
from osgeo import ogr
import numpy as np
import s3fs
import os
import os.path
import netCDF4

In [2]:
# Adjust the variable inputs to create the prefix for the file of interest
bucket_name = 'noaa-goes16' # Change to 'noaa-goes17' for GOES-17 satellite
product_name = 'ABI-L2-ACMC'# Choose a GOES-R L2+ product with a binary variable, 
                            #    ie. Clear Sky Mask (ACMC/ACMF) or Aersol Detection (ADPC/ADPF)
year = 2020 
day_of_year = 296 
hour = 16

# Adjust pathname to local directory
directory = "/Users/daniellelosos/GOESforGIS/"

In [3]:
# Make new directories to store files
if not os.path.isdir(directory + product_name +"_netCDFs"):
    os.mkdir(directory + product_name +"_netCDFs")
if not os.path.isdir(directory + product_name +"_shapefiles"):
    os.mkdir(directory + product_name +"_shapefiles") 

In [4]:
# Enable GDAL/OGR exceptions
gdal.UseExceptions()

# Use anonymous credentials to access public data  from AWS
fs = s3fs.S3FileSystem(anon=True)

# Write prefix for the files of interest, and list all files beginning with this prefix.
prefix = f'{bucket_name}/{product_name}/{year}/{day_of_year:03.0f}/{hour:02.0f}/'
print(prefix)
files = fs.ls(prefix)

noaa-goes16/ABI-L2-ACMC/2020/296/16/


In [5]:
#Read the first netCDF file in the list (closest to the top of the hour) and download
first_file = files[0]
path = directory + product_name +"_netCDFs/" + first_file.split("/")[-1]
fs.download(first_file, path)
dataset = gdal.Open(path, gdal.GA_ReadOnly)
print ("DATASET:\n" + first_file)
print ("\nSUBDATASETS:")
subdatasets = dataset.GetSubDatasets()
for fname, name in subdatasets:
    print (name)
    print (fname)

DATASET:
noaa-goes16/ABI-L2-ACMC/2020/296/16/OR_ABI-L2-ACMC-M6_G16_s20202961601166_e20202961603539_c20202961604353.nc

SUBDATASETS:
[1500x2500] cloud_binary_mask (8-bit integer)
NETCDF:"/Users/daniellelosos/GOESforGIS/ABI-L2-ACMC_netCDFs/OR_ABI-L2-ACMC-M6_G16_s20202961601166_e20202961603539_c20202961604353.nc":BCM
[1500x2500] status_flag (8-bit integer)
NETCDF:"/Users/daniellelosos/GOESforGIS/ABI-L2-ACMC_netCDFs/OR_ABI-L2-ACMC-M6_G16_s20202961601166_e20202961603539_c20202961604353.nc":DQF


In [6]:
# Choose the first variable/subdataset in the netCDF file to convert to a GeoTIFF.
# See the file's other variables in the "SUBDATASETS:" output, formatted as NETCDF:"filename":variable
# If the first subdataset does not contain the variable of intrest, adjust the list number below
f = netCDF4.Dataset(path)
var = list(f.variables.keys())[0] # (Optional) Adjust this list number, ie. [0] selects the 1st subdataset 
print ("Selected variable: " + var)

Selected variable: BCM


In [7]:
# Use gdal to open the selected variable's raster band
layer = gdal.Open("NETCDF:{0}:{1}".format(path, var))
band = layer.GetRasterBand(1)

# Create an output shapfile with a layer containing a new field
drv = ogr.GetDriverByName('ESRI Shapefile')
shp_name = directory + product_name + "_shapefiles" + var 
outfile = drv.CreateDataSource(shp_name + ".shp") 
outlayer = outfile.CreateLayer('polygonized raster', srs = None )
newField = ogr.FieldDefn(product_name, ogr.OFTReal)
outlayer.CreateField(newField)

# Use gdal.polygonize to convert the raster band into a polygons and store binary values of each polygon in new field
gdal.Polygonize(band, None, outlayer, 0, [])
outfile = None
print(shp_name + " has been generated")

/Users/daniellelosos/GOESforGIS/ABI-L2-ACMC_BCM has been generated
